In [5]:
import pandas as pd

In [68]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import csv
import json
import re
from tqdm import tqdm
from openai import AsyncClient
import asyncio

In [67]:
load_dotenv()

assert os.getenv("OPENAI_API_KEY") 

openai_client = AsyncClient()

In [8]:
def csv_converter(csv_file, jsonl_file):
    with open(csv_file, 'r', encoding='utf-8') as csv_file:
        csv_reader = csv.DictReader(csv_file)

        with open(jsonl_file, 'w', encoding='utf-8') as jsonl_file:
            for row in csv_reader:
                json_line = json.dumps(row, ensure_ascii=False)
                jsonl_file.write(json_line + '\n')

csv_converter("data.csv", "converted_data")

In [ ]:
async def generate_response_cot(messages, data_json):

    response = await openai_client.chat.completions.create(
    model="o4-mini-2025-04-16",
    messages = messages,
    response_format= {"type": "json_object"},
    )

    try:
        parsed_response_content = json.loads(response.choices[0].message.content)
    except TypeError as e:
        return False, None, None
    output_value = parsed_response_content.get("answer")
    cot_ans = parsed_response_content.get("cot")
    
    if (output_value == data_json["original_key"]) and (output_value == data_json["validator_answer"]):
        return True, output_value, cot_ans
    else:
        return False, output_value, cot_ans


In [93]:
async def check_answer_cot(background_info):
    out_answer = []
    cot_answers = []
    binary_answer = []
    choice_a = ""
    choice_b = ""
    pattern = r'"(.*?)"'

    with open(r"C:\Users\tolly\Desktop\MARS\nyu-debate-modeling\task_relevant\converted_data", "r", encoding="utf-8") as f:
        for line in tqdm(f, desc="Processing"):
            data_json = json.loads(line.strip())

            choice_a = data_json.get("choice_a")
            choice_b = data_json.get("choice_b")
            sentence = data_json.get("sentence")


            # lines = data_json["prompt"].splitlines()

            # for l in lines:
            #     if l.startswith("(A)"):
            #         choice_a = l
            #     elif l.startswith("(B)"):
            #         choice_b = l
    
            # matches = re.findall(pattern, data_json["prompt"])

            prompt = f"""
            Consider the English sentence "{sentence}" 
            Which of the following is a better translation of this sentence into standard Lojban? 
            {choice_a}  
            {choice_b} 
            Exactly one of the answers above is correct. Please provide your response in JSON format, including an 'answer' field for the final result and a 'cot' (chain of thought) field explaining your reasoning.

            Example JSON format:
            {{
                "answer": "Your final answer here",
                "cot": "Your step-by-step reasoning process here."
            }}
            """

            prompt_1 = f"""
            Consider the English sentence "{sentence}" 
            Which of the following is a better translation of this sentence into standard Lojban? 
            {choice_a}  
            {choice_b} 
            Exactly one of the answers above is correct. Please provide your response in JSON format, including an 'answer' (A or B) field for the final result and a 'cot' (chain of thought) field explaining your reasoning.

            Example JSON format:
            {{
                "answer": "Your final answer here",
                "cot": "Your step-by-step reasoning process here."
            }}

            Please use the background information provided below to help you answer given question:
            {background_info}
            """
    
            messages=[
                {"role": "system", "content": "You are an expert Lojban speaker"},
                {"role": "user", "content": f"{prompt_1}"},
            ]
            # b_ans, output_model, cot_model = generate_response_cot(messages, data_json)
            # print(generate_response_cot(messages, data_json))
            b_ans, out_ans, cot_ans = await generate_response_cot(messages, data_json)
            # correct_answers.append(output_model)
            cot_answers.append(cot_ans)
            binary_answer.append(b_ans)
            out_answer.append(out_ans) 
            
    return binary_answer, out_answer, cot_answers


In [38]:
with open("background_test.txt", "r", encoding="utf-8") as f:
    background_content = f.read()

In [76]:
final_binary, final_out, final_cot = check_answer_cot(background_content)

Processing: 0it [00:00, ?it/s]


RuntimeError: asyncio.run() cannot be called from a running event loop

In [40]:
df = pd.DataFrame({
    "Binary Answers": final_binary,
    "Final Answers": final_out,
    "CoT Answers": final_cot
})

In [47]:
print(final_cot[0])

In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.


In [65]:
for val, indx in zip((df[df["Binary Answers"] == False]["CoT Answers"].to_list()), (df.index[df["Binary Answers"] == False].to_list())):
    print(f"\nCoT Answer for jbo_{indx + 1}:")
    print(val)


CoT Answer for jbo_1:
In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.

CoT Answer for jbo_3:
We need to see what finti and fi do.   

finti   = x1 gives/makes x2 to x3    (without payment)  
se finti   swaps x1↔x2 so that x1=what was made  
fi        tags the next sumti as filling x3  

(A) lo pixra cu se finti lo ka jai spaji lo vitke  
    → se

In [52]:
for indx, val in enumerate(final_cot):
    print(f"\nCoT Answer {indx + 1}:")
    print(val)


CoT Answer 1:
In Lojban there are two common predicates for “think that …” and “believe that …”: mi pensi lo du‘u and mi jinvi lo du‘u. mi pensi lo du‘u expresses a tentative thought or opinion (“I think that…”), whereas mi jinvi lo du‘u is stronger—“I hold as true that…,” equivalent to English “I believe that.”

In English “I think we need to hurry up” uses “think” as a hedge or opinion marker, not a firm statement of belief. Therefore the Lojban equivalent should use pensi, not jinvi. Thus the better translation is:

mi pensi lo du‘u mi’a nitcu lo ka sutryze’a

The other choice (“mi jinvi lo du‘u …”) would mean “I believe that we need to hurry,” which is a stronger assertion than the English sentence.

CoT Answer 2:
1. The English jufra ‘Did you drink apple tea?’ is a yes/no question about drinking a particular beverage (‘apple tea’).
2. In Lojban the basic bridi for drinking is “do pinxe X” + question tag “xu”.
3. We must express ‘apple tea’ as a selbri + sumti or as a tanru. The w

In [43]:
sum(final_binary)/len(final_binary)

0.6209677419354839

In [ ]:
true_answer_cot = sum(final_binary)
true_answer_cot/len(final_answer_cot)

0.4032258064516129

In [ ]:
final_binary_non_cot, final_non_cot = check_answers()
sum(final_binary_non_cot)/len(final_binary_non_cot)

0.4838709677419355

In [ ]:
with open("background_test.txt", "r", encoding="utf-8") as f:
    background_content = f.read()


check_answer_cot(background_content)

In [66]:
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

def n_gram_coms(text, n):
    words = word_tokenize(text)
    combined_ngrams = [" ".join(gram) for gram in ngrams(words, n)]
    return combined_ngrams

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/tollymon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
from nltk.corpus import 

In [11]:
test1 = "ca lo nu snimycarvi kei mi dansu djica"

bigrams = n_gram_coms(test1, 2)

In [12]:
for b in bigrams:
    print(b)

('ca', 'lo')
('lo', 'nu')
('nu', 'snimycarvi')
('snimycarvi', 'kei')
('kei', 'mi')
('mi', 'dansu')
('dansu', 'djica')


In [10]:
import requests
from bs4 import BeautifulSoup
import os
import html2text
import re
import pandas as pd
import tiktoken


In [ ]:
url = "https://lojban.org/publications/cll/cll_v1.1_xhtml-section-chunks/"

def extract_link(url):
    source_url = requests.get(url)
    soup = BeautifulSoup(source_url.content, "html.parser")
    links = soup.find_all("a", href=True)

    return soup

In [35]:
def download_section_links(base_url, folder_name="downloaded_sections"):
    response = requests.get(base_url)
    response.raise_for_status()  #
    html_content = response.text
    soup = BeautifulSoup(html_content, "html.parser")
    links = soup.find_all("a", href=True)

    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    for link in links:
        href = link.get("href")
        if href and "section" in href:
            full_url = requests.compat.urljoin(base_url, href)
            response = requests.get(full_url)
            response.raise_for_status() 
            downloaded_html = response.text

            h = html2text.HTML2Text()
            h.ignore_links = True      
            h.ignore_images = True     
            h.body_width = 0           
            h.unicode_snob = True      # Handle Unicode characters better
            h.skip_internal_links = True # Skip anchors within the same document
            h.ignore_tables = False    # You might want to ignore tables too if structure is not needed
            h.single_line_break = True # Treat <br> as single newline

            plain_text = h.handle(downloaded_html)


            clean_href = re.sub(r'[#?].*$', '', href)
            base_filename = os.path.basename(clean_href)            
            file_name_without_ext = os.path.splitext(base_filename)[0]
            output_path = os.path.join(folder_name, f"{file_name_without_ext}.txt")

            with open(output_path, "w", encoding="utf-8") as f:
                f.write(plain_text)
            print(f"Saved: {file_name_without_ext}.txt")


In [37]:
download_section_links(url)

Saved: chapter-about.txt
Saved: section-what-is-cll.txt
Saved: section-conventions.txt
Saved: section-disclaimers.txt
Saved: section-bibliography.txt
Saved: section-captions.txt
Saved: section-copyright.txt
Saved: chapter-tour.txt
Saved: section-pronunciation.txt
Saved: section-sumti-cmavo.txt
Saved: section-some-selbri.txt
Saved: section-some-simple-bridi.txt
Saved: section-variant-bridi-structure.txt
Saved: section-order-of-sumti.txt
Saved: section-structure-of-utterances.txt
Saved: section-basic-tanru.txt
Saved: section-description-sumti.txt
Saved: section-some-brivla.txt
Saved: section-dihu-and-lahe-dihu.txt
Saved: section-possession.txt
Saved: section-vocatives-and-commands.txt
Saved: section-basic-questions.txt
Saved: section-attitudinals.txt
Saved: section-tenses.txt
Saved: section-terms.txt
Saved: chapter-phonology.txt
Saved: section-basic-phonetics.txt
Saved: section-lojban-characters.txt
Saved: section-diphthongs.txt
Saved: section-vowel-pairs.txt
Saved: section-clusters.txt


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
def get_top_tfidf_files_for_words(folder_path, words_of_interest, top_n=3):
    documents = []
    filenames = []

    enc = tiktoken.get_encoding("cl100k_base")
    file_contents_map = {}
    print(f"Reading files from: {folder_path}")

    # 1. Read all text files from the folder
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"): # Ensure we only read text files
            filepath = os.path.join(folder_path, filename)
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    content = f.read()
                    if content.strip(): # Only add non-empty documents
                        documents.append(content)
                        filenames.append(filename)
                        file_contents_map[filename] = content
                    else:
                        print(f"  Skipping empty file: {filename}")
            except Exception as e:
                print(f"  Error reading file {filename}: {e}")

    if not documents:
        print("No valid text files found or read in the specified folder.")
        return {}

    print(f"Successfully read {len(documents)} text files.")

    # 2. Initialize and Fit TfidfVectorizer
    # You can add parameters like stop_words='english', min_df, max_df etc.
    # depending on your specific needs for vectorization.
    vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b[\w']+\b")
    print("Fitting TfidfVectorizer and transforming documents...")
    tfidf_matrix = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out() # Get all words in the vocabulary
    print("Vectorization complete.")

    # Convert the sparse matrix to a DataFrame for easier indexing and sorting
    # Each row is a document, each column is a word from the vocabulary
    tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names, index=filenames)
    total_tokens = 0
    all_results_flat_list = []
    seen_combinations = set() 
    # 3. Calculate TF-IDF for specific words and find top N files
    print("\nCalculating top files for words of interest:")
    for word in words_of_interest:
        # Convert word to lowercase for consistent matching with vectorizer's vocabulary
        lower_word = word.lower()

        if lower_word in tfidf_df.columns:
            # Get the column of TF-IDF scores for the current word
            word_scores = tfidf_df[lower_word]

            # Sort the scores in descending order and get the top N files
            # .nlargest() is a convenient pandas method for this
            top_files_for_word = word_scores.nlargest(top_n)
            for filename, score in top_files_for_word.items():
                if score > 0: # Only include files with a non-zero TF-IDF score for this word
                    current_combination = (word, filename) 
                    if current_combination not in seen_combinations: 
                        file_content = file_contents_map.get(filename, "Content not found.") # Get content
                        total_tokens += len(enc.encode(file_content))
                        all_results_flat_list.append({
                            'word': word,
                            'filename': filename,
                            'tfidf_score': score,
                            'content': file_content
                        })
                        seen_combinations.add(current_combination) 
    return all_results_flat_list

In [12]:
test2 = "mi pensi lo du'u mi'a nitcu lo ka sutryze'a"
test3 = "mi jinvi lo du'u mi'a nitcu lo ka sutryze'a"
test_list = test2.split(" ")

In [95]:
test_list

['mi', 'jinvi', 'lo', "du'u", "mi'a", 'nitcu', 'lo', 'ka', "sutryze'a"]

In [23]:
target_folder = "downloaded_sections"
my_words = test_list

tfidf_analysis_results = get_top_tfidf_files_for_words(target_folder, my_words, top_n=1)

tfidf_analysis_results


# if tfidf_analysis_results:
#     for word, files_data in tfidf_analysis_results.items():
#         print(f"\nTop {3} files for word: '{word}'")
#         if isinstance(files_data, dict):
#             if files_data:
#                 for filename, score in files_data.items():
#                     print(f"  - {filename}: TF-IDF Score = {score:.6f}")
#             else:
#                 print("  No files found with a non-zero TF-IDF score for this word.")
#         else:
#             print(f"  {files_data}") # Message if word not in vocabulary
# else:
#     print("No TF-IDF results generated.")


Reading files from: downloaded_sections
Successfully read 318 text files.
Fitting TfidfVectorizer and transforming documents...
Vectorization complete.

Calculating top files for words of interest:
9113


[{'word': 'mi',
  'filename': 'section-more-propositions.txt',
  'tfidf_score': 0.2373682389679831,
  'content': 'Chapter 14. If Wishes Were Horses: The Lojban Connective System  \n---  \nPrev: Section 14.6 |  Next: Section 14.8  \n---|---  \nTable of Contents\nBook Info Page\n* * *\n## 14.7. More than two propositions\nSo far we have seen logical connectives used to connect exactly two sentences. How about connecting three or more? Is this possible in Lojban? The answer is yes, subject to some warnings and some restrictions.\nOf the four primitive truth functions A, E, O, and U, all but O have the same truth values no matter how their component sentences are associated in pairs. Therefore,\n**Example 14.31. **\nmi | dotco | .ije | mi | ricfu | .ije | mi | nanmu  \n---|---|---|---|---|---|---|---  \nI | am-German. | And | I | am-rich. | And | I | am-a-man.  \n  \n\nmeans that all three component sentences are true. Likewise,\n**Example 14.32. **\nmi | dotco | .ija | mi | ricfu | .ija |

In [24]:
def create_jsonl_defs(file_name, data_list):
    """
    Creates the jsonl for each of the type.
    """
    with open(file_name, 'w', encoding='utf-8', newline="") as f:
        for item in data_list:
            json_line = json.dumps(item, ensure_ascii=False)
            f.write(json_line + "\n")

In [57]:
import json

In [58]:
tfidf_analysis_results

[{'word': 'mi',
  'filename': 'section-more-propositions.txt',
  'tfidf_score': 0.25071599458056204,
  'content': 'Chapter 14. If Wishes Were Horses: The Lojban Connective System  \n---  \nPrev: Section 14.6 |  Next: Section 14.8  \n---|---  \nTable of Contents\nBook Info Page\n* * *\n## 14.7. More than two propositions\nSo far we have seen logical connectives used to connect exactly two sentences. How about connecting three or more? Is this possible in Lojban? The answer is yes, subject to some warnings and some restrictions.\nOf the four primitive truth functions A, E, O, and U, all but O have the same truth values no matter how their component sentences are associated in pairs. Therefore,\n**Example 14.31. **\nmi | dotco | .ije | mi | ricfu | .ije | mi | nanmu  \n---|---|---|---|---|---|---|---  \nI | am-German. | And | I | am-rich. | And | I | am-a-man.  \n  \n\nmeans that all three component sentences are true. Likewise,\n**Example 14.32. **\nmi | dotco | .ija | mi | ricfu | .ija 

In [25]:
create_jsonl_defs("lojban_grammar_search", tfidf_analysis_results)

In [ ]:
# some of the sections make sense, but not fundamentally a lot
# these are just to most relevant sections from the whole book I guess
# the hope is that these small parts will actually improve the way the output

In [76]:
from bs4 import BeautifulSoup
import re
import requests
import os
import json
import pandas as pd

In [77]:
new_url = "https://lojban.pw/en/texts/dictionary-with-examples/"

def getHTML(url):
    response = requests.get(url)
    return response.text

soup1 = BeautifulSoup(getHTML(new_url), 'html.parser')

column_classes = [
    'column-class-jbovla',
    'column-class-lojbo',
    'column-class-glico',
    'column-class-English_definition'
]

data = []
all_rows = soup1.find_all('tr')

for row in all_rows:
    row_data = {}
    for col_class in column_classes:
        # Find the td with the specific class within the current row
        cell = row.find('td', class_=col_class)
        if cell:
            # Extract text and strip whitespace
            row_data[col_class.replace('column-class-', '')] = cell.get_text(strip=True)
        else:
            # If a cell for this column class is not found, use an empty string or NaN
            row_data[col_class.replace('column-class-', '')] = None # or '' for empty string
    data.append(row_data)


In [78]:
def create_jsonl_defs(file_name, data_list):

    with open(file_name, 'w', encoding='utf-8', newline="") as f:
        for item in data_list:
            json_line = json.dumps(item, ensure_ascii=False)
            f.write(json_line + "\n")

create_jsonl_defs("dictionary_examples_data.jsonl", data)
pd.DataFrame(data)

,jbovla,lojbo,glico,English_definition
0,None,None,None,None
1,valsi fi le lojbo,jufra,English translation,definition of the Lojban word
2,liste lo'i xamgu jufra,xamgu jufra,Dictionary with Examples,
3,la gleki e lo drata,la gleki e lo drata,Gleki and others,
4,fanva fo le lojbo,fanva fo le lojbo,Translation from Lojban,
...,...,...,...,...
2725,zutse,e'a do zutse le stizu,You may sit down on the chair.,x1 sits [assumes sitting position] on surface ...
2726,zvafa'i,mi pu zvafa'i le mi'u penbi tu noi kumfa,I found the pen in that room.,
2727,zvati,la .alis. cu zvati ma,Where ia Alice?,x1 (object / event) is at / attending / presen...
2728,zvati,le sivni kumfa cu zvati ma,Where is the toilet?,x1 (object / event) is at / attending / presen...


In [1]:
with open("lojban_grammar_search", 'r', encoding='utf-8') as f:
    for line_num, line in enumerate(f):
        json_object = json.loads(line)
        print(json_object)


FileNotFoundError: [Errno 2] No such file or directory: 'lojban_grammar_search'

In [19]:
import pandas as pd
import json

In [3]:
data = pd.read_csv("original_validation_report_o3-2025-04-16.csv")

In [11]:
output = data[data["filename"]=="jbo_1.txt"]["best_match_in_background"].to_list()

In [65]:
formatted_list = [f"<verified>{str(i)}</verified>" for i in output]

In [66]:
formatted_string = "\n".join(formatted_list)

In [67]:
formatted_string

'<verified>{"Lojban": "pensi", "English": "- think; on a topic", "Type": "gismu", "Def": "x_{1} thinks/considers/cogitates/reasons/is pensive about/reflects upon subject/concept x_{2}."},</verified>\n<verified>{"Lojban": "jinvi", "English": "- opine", "Type": "gismu", "Def": "x_{1} thinks/opines x_{2} [opinion] (du\'u) is true about subject/issue x_{3} on grounds x_{4}."},</verified>'

In [1]:
a = "this is a test"
b = "I am a test"

set(a) - set(b)

{'h', 'i'}

In [8]:
a = "this is a test"
b = "I am a test"

set(a.split()).intersection(set(b.split()))

{'a', 'test'}

In [ ]:
set(a.split())

{'a', 'is', 'test', 'this'}

In [12]:
set(b.split())

{'I', 'a', 'am', 'test'}

In [16]:
# get unique items of each list, and when overlapping only one item

set([*a.split(), *b.split()])

{'I', 'a', 'am', 'is', 'test', 'this'}

In [17]:
import yaml

with open(r"C:\Users\tolly\Desktop\MARS\nyu-debate-modeling\prompts\configs\prompts.yaml", "r") as f:
    prompts = yaml.safe_load(f)
    

In [46]:
prompts["Debate Prompt"]["debater_system"]["content"][1]

'You are <NAME>, an AI who is trying to win a debate as part of an educational activity. \n  - You keep your arguments brief whenever possible.\n  - You try to defend your assigned side, even if you might personally disagree.\n  - Please cite the background text by wrapping direct quotes with <quote></quote> tags.\n'

In [37]:
prompts.get("judge_system")

In [33]:
test.replace("<NAME>", "Test")

'You are Test, an intelligent AI with the goal of winning the debate. \n  - You are concise and clearly state your arguments.\n  - Every statement you make is supported by evidence and logical reasoning. \n  - Please cite the background text by wrapping direct quotes with <quote></quote> tags.\n'

In [8]:
import csv
import json
import os

In [ ]:
def csv_converter_final(csv_file, jsonl_file, columns, tuple_columns=None, tuple_field_name=None, prompt_folder=None, prompt_field_name="prompt_file_content"):
    with open(csv_file, 'r', encoding='utf-8') as f_in, open(jsonl_file, 'w', encoding='utf-8') as f_out:
        reader = csv.DictReader(f_in)
        for row in reader:
            filtered_row = {k: row[k] for k in columns if k in row}
            if tuple_columns and tuple_field_name:
                filtered_row[tuple_field_name] = tuple(row[col] for col in tuple_columns if col in row)
                for col in tuple_columns:
                    filtered_row.pop(col, None)

            if prompt_folder and "original_id" in filtered_row:
                file_path = os.path.join(prompt_folder, f"background_{filtered_row['original_id']}.txt")
                try:
                    with open(file_path, 'r', encoding='utf-8') as pf:
                        filtered_row[prompt_field_name] = pf.read()
                except Exception as e:
                    filtered_row[prompt_field_name] = f"File not found: {file_path}"

            json_line = json.dumps(filtered_row, ensure_ascii=False)
            f_out.write(json_line + '\n')

csv_converter_final(
    "data.csv",
    "lojban_dataset.jsonl",
    columns=["original_id", "prompt", "original_key","choice_a", "choice_b"],
    tuple_columns=["choice_a", "choice_b"],
    tuple_field_name="answers",
    prompt_folder=r"c:\Users\tolly\Desktop\MARS\nyu-debate-modeling\task_relevant\background_files",
    prompt_field_name="prompt_file_content"
)